In [131]:
import pandas as pd
import numpy as np

# User Monitoring Pipeline:

### Create Tables in the AWS instance

In [132]:
# Use the following SQL code in MySQL Workbench in order to create the following tables in the User_Monitoring database:
# 1) ucs: two fields [uuid, score] --> [user's unique identifier, and their current UCS score]
# 2) task_scores: four fields [ts, quiz_task_uuid, user_uuid, task_score] -->
#                             [time stamp, task identifier, user's unique identifier, task_score]
# 3) datahunt_tracker: two fields [datahunt_id, num_rows_processed] --> 
#                                 [datahunt's unique id, number of rows in the datahunt that we processed already]
"""

CREATE DATABASE User_Monitoring;

USE User_Monitoring;

CREATE TABLE ucs (
	uuid INT,
    score DECIMAL(6,5) NOT NULL
);

desc UCS;

SELECT * FROM UCS;

----------------
CREATE TABLE task_scores (
	ts TIMESTAMP,
    quiz_task_uuid INT,
    user_uuid INT,
    task_score DECIMAL(6,5)
);

desc task_scores;

SELECT * FROM task_scores;


----------------
CREATE TABLE datahunt_tracker (
	datahunt_id INT,
    num_rows_processed INT
);

desc datahunt_tracker;

SELECT * FROM datahunt_tracker;


"""

'\n\nCREATE DATABASE User_Monitoring;\n\nUSE User_Monitoring;\n\nCREATE TABLE ucs (\n\tuuid INT,\n    score DECIMAL(6,5) NOT NULL\n);\n\ndesc UCS;\n\nSELECT * FROM UCS;\n\n----------------\nCREATE TABLE task_scores (\n\tts TIMESTAMP,\n    quiz_task_uuid INT,\n    user_uuid INT,\n    task_score DECIMAL(6,5)\n);\n\ndesc task_scores;\n\nSELECT * FROM task_scores;\n\n\n----------------\nCREATE TABLE datahunt_tracker (\n\tdatahunt_id INT,\n    num_rows_processed INT\n);\n\ndesc datahunt_tracker;\n\nSELECT * FROM datahunt_tracker;\n\n\n'

### Create handler functions to interact with AWS instance

In [140]:
# Install pymysql in the current Jupyter kernel
import sys
# !conda install --yes --prefix {sys.prefix} pymysql

import pymysql
import time
# 1. Install pymysql to local directory
# pip install -t $PWD pymysql

# 2. Write code, then zip it up

# Lambda Permissions:
# AWSLambdaVPCAccessExecutionRole

# Configuration Files
endpoint = 'user-monitoring-database.crnwwfmibeif.us-west-1.rds.amazonaws.com'
username = 'admin'
password = 'user_monitoring'
database_name = 'User_Monitoring'

# Connection
connection = pymysql.connect(
    host=endpoint, user=username, passwd=password, db=database_name)
cursor = connection.cursor()

def lambda_handler(event=None, context=None):
    insert_into_table(event['table'])

def table_to_df(table):
    cursor = connection.cursor()
    cursor.execute('SELECT * from {}'.format(table))
    rows = cursor.fetchall()
    field_names = [i[0] for i in cursor.description]
    df = pd.DataFrame(columns=field_names)
    for row in rows:
        df.loc[len(df.index)] = row
    return df

def display_table(table):
    df = table_to_df(table)
    print(table)
    display(df)

def table_to_csv(table):
    df = table_to_df(table)
    df.to_csv(f'{table}.csv', index=False)

def insert_into_table(table, df):
    """
    TODO: modify function to be dynamic. take in table name to insert into as well
          as dataframe to add into the selected table. add data validation to make sure
          df is correctly formatted for table
    """
    cursor = connection.cursor()
    mysql_query = None
    if table == "ucs":
        # df columns: 'contributor_uuid, score'
        
        insert_ucs = "INSERT INTO `ucs` (`uuid`, `score`) VALUES (%s, %s)"
        def ucs_query(row):
            data_ucs = (row['contributor_uuid'], row['score'])
            cursor.execute(insert_ucs, data_ucs)
            return row
        mysql_query = ucs_query
        
        # cursor.execute('SELECT * from ucs')
    elif table == "task_scores":
        # df columns: 'quiz_task_uuid, contributor_uuid, score'
        
        insert_task_scores = "INSERT INTO task_scores (ts, quiz_task_uuid, user_uuid, task_score) VALUES (%s, %s, %s, %s)"  
        def task_scores_query(row):
            ts = time.strftime("%Y-%m-%d %H:%M:%S")
            quiz_task_uuid = row['quiz_task_uuid']
            contributor_uuid = row['contributor_uuid']
            score = row['score']
            data_task_scores = (ts, quiz_task_uuid, contributor_uuid, score)
            cursor.execute(insert_task_scores, data_task_scores)
            return row
        mysql_query = task_scores_query
        
        # cursor.execute('SELECT * from task_scores')
    elif table == "datahunt_tracker":
        # df columns: 'datahunt_id, num_rows_processed'
        
        insert_datahunt_tracker = "INSERT INTO datahunt_tracker (datahunt_id, num_rows_processed) VALUES (%s, %s)"
        def datahunt_tracker_query(row):
            datahunt_id = row['datahunt_id']
            num_rows_processed = row['num_rows_processed']
            data_datahunt_tracker = (datahunt_id, num_rows_processed)
            cursor.execute(insert_datahunt_tracker, data_datahunt_tracker)
            return row
        
        # cursor.execute('SELECT * from datahunt_tracker')
    
    # run the appropriate query on each row of the given dataframe
    df = df.apply(mysql_query, axis=1)
    connection.commit()
    
    display_table(table)

In [141]:
display_table('task_scores')

task_scores


ts  quiz_task_uuid  user_uuid task_score
0 2022-04-02 17:59:18               1         10    0.60000
1 2022-04-02 17:59:18               2         20    0.80000

In [142]:
table_to_csv('task_scores')

### Creating Task Scores
This part of the notebook will include steps 2 and 3 from the User Monitoring Pipeline, which includes identifying the consensus answer from the IAA and Gold Standard data (step 2) and creating the corresponding task scores for users who completed this task (step 3).

## Hardcoded Evidence Schema
Most of this information will be in some sort of schema file (see file 'Evidence2021_05_19-Schema.csv'), but I'm not sure where the schema file is for this specific set of tasks. Thus, I hard coded it with the schema data from https://github.com/Goodly/PEUserMonitoring/blob/master/task-schema/Evidence.txt. Getting this information with the right schema file should be fairly straightforward.


__TODO__: Get schema file and implement method to retrieve the set of scored questions as well as a nested dictionary represented the question schema.

In [40]:
scored_questions = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

question_schema = {1:{'type':'select_one_nominal', 'num_choices':3},
           2:{'type':'select_all', 'num_choices':9},
           3:{'type':'select_one_nominal', 'num_choices':1},
           4:{'type':'select_one_ordinal', 'num_choices':6},
           5:{'type':'select_one_nominal', 'num_choices':5},
           6:{'type':'select_one_nominal', 'num_choices':3},
           7:{'type':'select_one_ordinal', 'num_choices':1},
           8:{'type':'select_one_ordinal', 'num_choices':5},
           9:{'type':'select_one_ordinal', 'num_choices':3},
           10:{'type':'select_one_ordinal', 'num_choices':5},
           11:{'type':'select_one_ordinal', 'num_choices':5},
           12:{'type':'select_one_ordinal', 'num_choices':4},
           13:{'type':'select_one_ordinal', 'num_choices':10},
           14:{'type':'select_one_ordinal', 'num_choices':10}}

## Preprocessing of IAA and Gold Standard Data

In [41]:
# read in the data
adjudicated = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidence2020_03_21.adjudicated-edb1510f-1923-4d6f-a678-95f53d752bea-Tags.csv')
iaa = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1.IAA-edb1510f-1923-4d6f-a678-95f53d752bea-Tags.csv')

# getting rid of some rows where the answer was invalid, probably represents some other metadata
iaa = iaa[iaa.answer_uuid.str.len() > 3]

In [42]:
# these are the only relevant columns for scoring for now, notice highlight data is not included here
cols = ['answer_uuid', 'question_Number', 'agreed_Answer']

# getting rid of some rows where the above columns were the same, this may represent different 
# highlights for the same question and answer?
adjudicated = adjudicated[cols].drop_duplicates()
iaa = iaa[cols].drop_duplicates()

Below cells just show the format of the preprocessed IAA and Adjudicated / Gold Standard data.

In [43]:
adjudicated.head(3)

answer_uuid  question_Number  agreed_Answer
0  73d7a14a-9ec6-404c-b2b7-a55508af3b76                1              1
1  5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f                2              1
3  ba2d1638-2509-4ce8-9130-39ea26d1d424                2              2

In [44]:
iaa.head(3)

answer_uuid  question_Number agreed_Answer
0  73d7a14a-9ec6-404c-b2b7-a55508af3b76                1             1
3  5a1fb1f4-d8b7-45c0-bce5-7d4c3b91c55f                2             1
4  ba2d1638-2509-4ce8-9130-39ea26d1d424                2             2

## Creating The Answer Key

In [45]:
# consensus answer key
consensus_answers = {}

def get_answer(question, answer_source):
    """
    Take in the question and the answer_source, either IAA or Adjudicated / Gold Standard, and adds the
    converged consensus answer to the consensus_answer answer key. This will be an single
    int for select_one questions, or a list of ints for select_all questions.
    """
    question_type = question_schema[question]['type']
    
    if question_type == 'select_one_nominal' or question_type == 'select_one_ordinal':
        assert len(answer_source[answer_source.question_Number == question].agreed_Answer) == 1
        consensus_answers[question] = answer_source[answer_source.question_Number == question].agreed_Answer.iloc[0]
    elif question_type == 'select_all':
        consensus_answers[question] = list(answer_source[answer_source.question_Number == question].agreed_Answer)
    else:
        raise ValueError('Invalid question type')

In [46]:
# create a set of questions that the Adjudicated / Gold Standard data determined converged to a consensus
adjudicated_consensus_questions = set(adjudicated.question_Number)

# create a set of questions that the IAA data determined converged to a consensus
iaa_consensus_questions = set(iaa.question_Number)

# uses get_answer function to fill in the consensus_answers answer key
for question in scored_questions:
    if question in adjudicated_consensus_questions:
        get_answer(question, adjudicated)
    elif question in iaa_consensus_questions:
        get_answer(question, iaa)
    else:
        consensus_answers[question] = -1

What the consensus key looks like:

In [47]:
consensus_answers

{1: 1,
 2: [1, 2, 3, 5],
 3: -1,
 4: 2,
 5: 5,
 6: '3',
 7: 1,
 8: 4,
 9: 1,
 10: 4,
 11: 4,
 12: 2}

## Scoring Users

In [48]:
def scoring_select_one_nominal(question, answer):
    """
    Takes in a question and the selected answer, returns a score of 0 if the consensus 
    answer is different, and 1 if the consensus answer is the same.
    """
    consensus_answer = consensus_answers[question]
    return int(consensus_answer == answer)

In [49]:
def scoring_select_one_ordinal(question, answer):
    """
    Takes in a question and the selected answer, returns a score between 0 and 1 depending
    on how far off the answer is from the consensus answer.
    """
    consensus_answer = consensus_answers[question]
    num_choices = question_schema[question]['num_choices']
    return 1 - (abs(answer - consensus_answer) / num_choices)

In [50]:
def scoring_select_all(question, answer_list):
    """
    Takes in a question and the selected answer, returns a score between 0 and 1 depending
    on the accuracy ((True Positive + True Negative) / Total) of the answer selections
    compared to the consensus answer selections.
    """
    answer_set = set(answer_list)
    consensus_answer_set = set(consensus_answers[question])
    num_choices = question_schema[question]['num_choices']
    
    total_correct = 0
    for answer in range(1, num_choices+1):
        if (answer in answer_set) and (answer in consensus_answer_set):
            total_correct += 1
        elif (answer not in answer_set) and (answer not in consensus_answer_set):
            total_correct += 1
        else:
            total_correct += 0
        
    return total_correct / num_choices

In [51]:
def scoring(row):
    """
    This is a Pandas apply function, to be applied on axis=1 (on each row).
    Makes a call to one of scoring_select_one_nominal, scoring_select_one_ordinal, and
    scoring_select_all depending on the type of question, returns the outputted score.
    
    An important note is that right now if neither IAA nor Gold Standard have a consensus
    answer for a question, the consensus_answers answer key will contain a -1 for that 
    question. I currently assume this question should not have been answered due to it
    being a child-question from an incorrectly answered parent question, so I score it
    """
    question = int(row['question_label'])
    answer_list = [int(i) for i in row['answer_label']]
    
    if consensus_answers[question] == -1:
        return 0
    
    question_type = question_schema[question]['type']
    if question_type == 'select_one_nominal':
        return scoring_select_one_nominal(question, answer_list[0])
    elif question_type == 'select_one_ordinal':
        return scoring_select_one_ordinal(question, answer_list[0])
    elif question_type == 'select_all':
        return scoring_select_all(question, answer_list)
    else:
        raise ValueError('Invalid question type')

In [52]:
# read in the datahunt
df_full = pd.read_csv('evidence_eric/evidence_eric/Covid_Evidencev1-Task-2224-DataHunt.csv')

In [53]:
df_full

namespace  \
0    Covid_Evidence2020_03_21   
1    Covid_Evidence2020_03_21   
2    Covid_Evidence2020_03_21   
3    Covid_Evidence2020_03_21   
4    Covid_Evidence2020_03_21   
..                        ...   
768  Covid_Evidence2020_03_21   
769  Covid_Evidence2020_03_21   
770  Covid_Evidence2020_03_21   
771  Covid_Evidence2020_03_21   
772  Covid_Evidence2020_03_21   

                                         schema_sha256  \
0    45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
1    45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
2    45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
3    45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
4    45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
..                                                 ...   
768  45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
769  45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
770  45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
771  45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   
772  45dce5251bd3ea6e908fa33ac9e6a8e17e6830215912ce...   

                           quiz_task_uuid  \
0    edb1510f-1923-4d6f-a678-95f53d752bea   
1    edb1510f-1923-4d6f-a678-95f53d752bea   
2    edb1510f-1923-4d6f-a678-95f53d752bea   
3    edb1510f-1923-4d6f-a678-95f53d752bea   
4    edb1510f-1923-4d6f-a678-95f53d752bea   
..                                    ...   
768  edb1510f-1923-4d6f-a678-95f53d752bea   
769  edb1510f-1923-4d6f-a678-95f53d752bea   
770  edb1510f-1923-4d6f-a678-95f53d752bea   
771  edb1510f-1923-4d6f-a678-95f53d752bea   
772  edb1510f-1923-4d6f-a678-95f53d752bea   

                                              task_url  \
0    https://pe.goodlylabs.org/project/Covid_Eviden...   
1    https://pe.goodlylabs.org/project/Covid_Eviden...   
2    https://pe.goodlylabs.org/project/Covid_Eviden...   
3    https://pe.goodlylabs.org/project/Covid_Eviden...   
4    https://pe.goodlylabs.org/project/Covid_Eviden...   
..                                                 ...   
768  https://pe.goodlylabs.org/project/Covid_Eviden...   
769  https://pe.goodlylabs.org/project/Covid_Eviden...   
770  https://pe.goodlylabs.org/project/Covid_Eviden...   
771  https://pe.goodlylabs.org/project/Covid_Eviden...   
772  https://pe.goodlylabs.org/project/Covid_Eviden...   

                                 tua_uuid  \
0    a723537a-f11a-41dd-bf5b-668cef67a5de   
1    a723537a-f11a-41dd-bf5b-668cef67a5de   
2    a723537a-f11a-41dd-bf5b-668cef67a5de   
3    a723537a-f11a-41dd-bf5b-668cef67a5de   
4    a723537a-f11a-41dd-bf5b-668cef67a5de   
..                                    ...   
768  a723537a-f11a-41dd-bf5b-668cef67a5de   
769  a723537a-f11a-41dd-bf5b-668cef67a5de   
770  a723537a-f11a-41dd-bf5b-668cef67a5de   
771  a723537a-f11a-41dd-bf5b-668cef67a5de   
772  a723537a-f11a-41dd-bf5b-668cef67a5de   

                               article_batch_name  article_number  \
0    CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
1    CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
2    CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
3    CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
4    CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
..                                            ...             ...   
768  CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
769  CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
770  CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
771  CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   
772  CovidArticles/Covid_article_for_PE_S&S&S.txt          100059   

                   article_filename  \
0    Covid_article_for_PE_S&S&S.txt   
1    Covid_article_for_PE_S&S&S.txt   
2    Covid_article_for_PE_S&S&S.txt   
3    Covid_article_for_PE_S&S&S.txt   
4    Covid_article_for_PE_S&S&S.txt   
..                              ...   
768  Covid_article_for_PE_S&S&S.txt   
769  C

In [54]:
# narrow down the datahunt to the relevant columns for scoring, getting rid of some rows
# where the data for the below columns were the same, this may represent different highlights 
# for the same question and answer? not certain.
df = df_full[['contributor_uuid', 'question_label', 'answer_label']].drop_duplicates()

# the question and answer labels in the datahunt are in the form 'T1.QX' and 'T1.QX.AX'
# the below lines strip down to only question number and answer number
df['question_label'] = df['question_label'].str.split('Q').str[1].astype(int)
df['answer_label'] = df['answer_label'].str.split('A').str[1]

In [55]:
df

contributor_uuid  question_label answer_label
0    e1ae8875-a398-4dde-8f4e-4b21109784e3               1            1
1    e1ae8875-a398-4dde-8f4e-4b21109784e3               2            1
2    e1ae8875-a398-4dde-8f4e-4b21109784e3               2            2
3    e1ae8875-a398-4dde-8f4e-4b21109784e3               2            3
4    e1ae8875-a398-4dde-8f4e-4b21109784e3               4            6
..                                    ...             ...          ...
768  bd786026-bad5-4fa8-9a3a-38ca03a16412              10            5
769  bd786026-bad5-4fa8-9a3a-38ca03a16412              11            4
770  bd786026-bad5-4fa8-9a3a-38ca03a16412              12            2
771  bd786026-bad5-4fa8-9a3a-38ca03a16412              13            6
772  bd786026-bad5-4fa8-9a3a-38ca03a16412              14            9

[641 rows x 3 columns]

In [56]:
# we want to groupby contributor_uuid and question_label to get all the answers a user
# selected for a particular question, to account for select_all questions. Now, the
# granularity of df_grouped will be one row per contributor answering a question.
df_grouped = df.groupby(['contributor_uuid', 'question_label']).agg(list).reset_index()

In [57]:
df_grouped

contributor_uuid  question_label  answer_label
0    00f548b7-6b63-4b47-828e-8e416b6ca0e2               1           [1]
1    00f548b7-6b63-4b47-828e-8e416b6ca0e2               2  [3, 5, 8, 4]
2    00f548b7-6b63-4b47-828e-8e416b6ca0e2               3           [1]
3    00f548b7-6b63-4b47-828e-8e416b6ca0e2               4           [6]
4    00f548b7-6b63-4b47-828e-8e416b6ca0e2               5           [2]
..                                    ...             ...           ...
549  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              10           [4]
550  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              11           [4]
551  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              12           [2]
552  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              13           [2]
553  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              14           [7]

[554 rows x 3 columns]

In [58]:
# we only want to score the rows with scored questions (not survey questions like 13 and 14)
# so we'll filter those out
df_grouped = df_grouped[df_grouped.question_label.isin(scored_questions)]

In [59]:
df_grouped

contributor_uuid  question_label  answer_label
0    00f548b7-6b63-4b47-828e-8e416b6ca0e2               1           [1]
1    00f548b7-6b63-4b47-828e-8e416b6ca0e2               2  [3, 5, 8, 4]
2    00f548b7-6b63-4b47-828e-8e416b6ca0e2               3           [1]
3    00f548b7-6b63-4b47-828e-8e416b6ca0e2               4           [6]
4    00f548b7-6b63-4b47-828e-8e416b6ca0e2               5           [2]
..                                    ...             ...           ...
547  fd6f6837-7881-4943-8129-b7ea0f0fe1b6               8           [5]
548  fd6f6837-7881-4943-8129-b7ea0f0fe1b6               9           [1]
549  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              10           [4]
550  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              11           [4]
551  fd6f6837-7881-4943-8129-b7ea0f0fe1b6              12           [2]

[440 rows x 3 columns]

In [60]:
# using the scoring function defined above, we'll create a new column containing the scores
# for each contributor answering a question.
df_grouped['score'] = df_grouped.apply(scoring, axis=1)

<ipython-input-60-8dfab57a0f1b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grouped['score'] = df_grouped.apply(scoring, axis=1)


This is the current format of df_grouped.

In [61]:
df_grouped.head(3)

contributor_uuid  question_label  answer_label  \
0  00f548b7-6b63-4b47-828e-8e416b6ca0e2               1           [1]   
1  00f548b7-6b63-4b47-828e-8e416b6ca0e2               2  [3, 5, 8, 4]   
2  00f548b7-6b63-4b47-828e-8e416b6ca0e2               3           [1]   

      score  
0  1.000000  
1  0.555556  
2  0.000000

In [62]:
# lastly, we want to get the average score for all task responses, this will be their
# task score. this is done by a simple groupby on contributor_uuid and mean() aggregate function
calculated_task_scores = df_grouped[['contributor_uuid', 'score']].groupby('contributor_uuid').mean()

This is what the final task_scores output will look like:

In [63]:
calculated_task_scores

score
contributor_uuid                              
00f548b7-6b63-4b47-828e-8e416b6ca0e2  0.603241
070268de-067c-463b-9ad3-5c88292d881e  0.878889
082a8363-a579-41b4-8918-c166fec3a3a4  0.513333
09df3ada-e5a8-4419-b78a-e0d1e9b37484  0.277778
09f279ac-1c34-4a84-8972-3d92b93605a7  0.361111
0c22ce7c-4641-4bb1-97f4-7a7355f70f25  0.831481
0e51ab2d-1a03-4d18-be33-fd21a829d19b  0.754630
1b69eee8-ab95-49dd-8979-9fff7655964d  0.277778
21ffd986-c219-43a0-b82f-4cc460da628d  0.831481
24640f45-b90b-40dc-a848-9e03fdfbbf91  0.588889
2658cecf-829f-4b91-84fa-333e1ff2533c  0.574537
2de6fa3e-ba77-4247-97a1-ce5661997618  0.361111
3254f476-8fe5-4b93-8ab3-0a858015fde1  0.597475
348c9e41-2d9c-47fa-b13c-3ec96d70afcc  0.817284
38843a0b-310a-4083-920e-4c7431c085d9  0.789899
3de829c9-4cc0-42b6-a29b-f4d8dcb861a3  0.690909
411f7dff-e994-433c-b5c6-a7d4adf9b010  0.361111
5059cbd7-64ee-48c0-8942-583f9e6c67c2  0.361111
51f195d5-1362-4a7e-9d72-3aa9ff3c437d  0.712222
62b8fd1c-9cd2-4a31-a8a1-0641d5b8dc71  0.736869
6d82de9d-0553-47b6-b678-d0110a069663  0.593750
6e035130-fe33-4c6b-8712-1aefc4bc68de  0.514815
7c99c1f8-6853-409c-a6c4-124bdfb6489f  0.753968
85579cf2-e01c-45c5-b9e7-34b40467148d  0.828571
8f5c408e-20c1-4f79-ac52-bb772dff6b32  0.361111
90f54517-9a8b-43d1-9ea1-95ba35b465d4  0.277778
95dc40d9-3710-47d7-abf0-24b825a1d0c5  0.890741
981874cf-9246-4e1e-b23c-00d432126c4f  0.875309
a1646881-e64e-4139-b608-f4d4a42de866  0.604938
a2edb56d-11a9-4823-93c7-260e11d36668  0.678395
a5a60ffa-d84a-46f1-9011-38515d1c249e  0.361111
ac49bd61-73d3-475b-9dcd-e8f476c0c19b  0.766667
ace2c639-8cd5-403e-8b0e-1af7faddf349  0.597475
b3d267dc-896f-4bc9-8f9b-a6a942db579a  0.636364
ba750de9-78de-45bb-a929-873704fe71e8  0.752525
bd786026-bad5-4fa8-9a3a-38ca03a16412  0.736364
c0697300-b1e0-48fd-92cd-958aa1b8a4f7  0.850000
c0ab7c63-2f39-405a-97cf-8a0e4161c03e  0.277778
c2ce666b-eb5f-416a-bfea-8125ff5606f3  0.670202
c3a4c4a5-1efe-4554-ab45-60f32737a52d  0.781818
cffa52fe-b6e2-433e-ba12-f36225f8ec5c  0.656566
d50137ad-8dce-4274-85c5-c2b399ce7ae0  0.277778
d5a1b3dc-4247-4f6f-8c2c-a11a4877cf01  0.767172
d77af5c1-a742-4613-9bb6-0260e4945899  0.733333
db8307ca-1d91-4749-b1e5-ac4bb8aba075  0.716162
dc33f0ea-1670-4a8e-80a0-b10aa78bba24  0.937778
de68bbf8-46d7-45f0-9111-92e64ab9499a  0.277778
e1ae8875-a398-4dde-8f4e-4b21109784e3  0.891358
e7042c9b-f64c-4a6a-a8c9-f2cc7fca6866  0.834568
e8542793-c51d-4edd-9f27-f2cb6005ed39  0.686420
eddb3434-036e-4901-a4cf-02b10326e7dc  0.896296
ee0c5dfb-4021-4d7f-b62a-f9830df90514  0.758333
f7ba7233-32f2-4390-b026-16acd43954f0  0.361111
f81fbbc0-feb7-454d-b486-c24e17bccad6  0.900617
f9143626-bfe0-4e69-b652-6d1525ab4eb0  0.361111
fc3f3244-1ff1-4e88-b1cd-51368ad5a313  0.933333
fd6f6837-7881-4943-8129-b7ea0f0fe1b6  0.963333

The below value is the number of rows processed in this datahunt, to be used for updating the datahunt tracking table.

In [64]:
rows_processed = len(df_full)
rows_processed

773

The below value represents an identifier for each datahunt, meaning an identifier for each task since each datahunt corresponds to one task.

In [65]:
quiz_task_uuid = df_full['quiz_task_uuid'][0]
quiz_task_uuid

'edb1510f-1923-4d6f-a678-95f53d752bea'

## Updating the User-Monitoring Database

In [71]:
# ucs function that reads accepts values from a csv and a current user's ucs score
def ucs_update_score(user_id, cur_ucs):
    def logistic(x, k, offset):
        return 1 / (1 + np.e**(-k * (x - offset)))
    # each row of the csv corresponds to one user, with the first value being the user_id and the rest being task scores
    def task_scores_from_csv(user_id):
        #      0   1   2   3
        # id1  11  12  13  14
        # id2  21  22  23  24
        # id3  31  32  33  34
        df = pd.read_csv('task_scores.csv', header=None, index_col=0)
        task_scores = df.loc[[user_id]].to_numpy()[0]
        return task_scores
    task_scores = task_scores_from_csv(user_id)
    a = 1000
    num_task_scores = len(task_scores)
    n = min(10, int(np.sqrt(num_task_scores)) + 1)
    if num_task_scores == 0:
        return 0.5
    else:
        var_scores = np.var(task_scores[-n:])
        c = logistic(var_scores / (np.log(num_task_scores + 1) / (np.log(a))), 10, 0.2)
        new_ucs = cur_ucs * (1 - c) + (c) * task_scores[-1] #task_scores[-1] = latest task score

    return new_ucs

#### Step 1) Insert calculated_task_scores into the task scores table and check that the task_scores table updated properly

#### Step 2) Update User Credibility Score:
Algorithm: Once per row in calculated_task_scores:
--> if contributor_uuid not in 